# Azure OpenAI and Llama-Index

In this quickstart you will create a simple Llama Index App and learn how to log it and get feedback on an LLM response using both an embedding and chat completion model from Azure OpenAI.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/models/azure_openai_llama_index.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
#! pip install trulens-eval==0.11.0 llama_index==0.8.3 langchain==0.0.248 html2text==2020.1.16

### Add API keys
For this quickstart, you will need a larger set of information from Azure OpenAI compared to typical OpenAI usage.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["OPENAI_API_BASE"] = "..."
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_TYPE"] = "azure"

### Import from TruLens

In [ ]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()

### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [ ]:
import os
import json
import openai
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import AzureOpenAI
from llama_index import LangchainEmbedding
from llama_index import VectorStoreIndex, SimpleWebPageReader, ServiceContext
from llama_index import set_global_service_context
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# get model from Azure
llm = AzureOpenAI(engine="...", model="text-davinci-003") 

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="...",
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_type=os.environ["OPENAI_API_TYPE"],
        openai_api_version=os.environ["OPENAI_API_VERSION"]
    ),
    embed_batch_size=1
)

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm
)

set_global_service_context(service_context)

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

### Send your first request

In [ ]:
query = "What is most interesting about this essay?"
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

## Initialize Feedback Function(s)

In [ ]:
import numpy as np
# Initialize AzureOpenAI-based feedback function collection class:
azopenai = feedback.AzureOpenAI(model_engine="gpt-35-turbo", deployment_id="...")

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(azopenai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(azopenai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.mean)

# groundedness of output on the context
groundedness = feedback.Groundedness(summarize_provider=azopenai, groundedness_provider=azopenai)
f_groundedness = Feedback(groundedness.groundedness_measure).on(TruLlama.select_source_nodes().node.text).on_output()

## Instrument chain for logging with TruLens

In [ ]:
tru_query_engine = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

In [ ]:
query = "What is most interesting about this essay?"
answer = tru_query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all